In [3]:
import pandas as pd
import json
import numpy as np
import datetime

In [4]:
ml_data = pd.read_csv('./raw_data/ml_data_last_interaction.csv',index_col=0)
ml_data

,other_created_address,date_creation_diff,verified,last_tx_date
0,0xfa7d2a996ac6350f4b56c043112da0366a59b74c,438,1,2023-02-06 14:39:11
1,0x575ccd8e2d300e2377b43478339e364000318e2c,0,1,2023-04-10 00:22:47
2,0x705350c4bcd35c9441419ddd5d2f097d7a55410f,-1,1,2023-04-01 11:19:11
3,0xa90996896660decc6e997655e065b23788857849,-1,1,2023-04-10 22:55:59
4,0xb1f2cdec61db658f091671f5f199635aef202cac,-1,1,2023-04-13 01:49:23
...,...,...,...,...
731,0xeee72c5da95a1e4bbcf0bf327518045ff5a2916f,149,0,2021-04-22 05:54:52
732,0xca658217ce94dfb2156a49a8fad0ff752cac39c2,59,0,2023-04-09 11:15:11
733,0x06a00715e6f92210af9d7680b584931faf71a833,0,1,2023-04-12 00:59:35
734,0xcafe001067cdef266afb7eb5a286dcfd277f3de5,0,1,2023-04-13 09:19:35


In [5]:
#cut off date: 2023-04-13
now_date = datetime.datetime(2023, 4, 13, 0, 0, 0)

'2023-02-06 14:39:11'

In [6]:

ml_data['last_tx_in_days'] = ml_data['last_tx_date'].apply(lambda x: (now_date- datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S")).days)

In [12]:
gas_feature_df = pd.read_csv('./raw_data/ml_data_tx_gas_and_initiator.csv',index_col=0)
gas_feature_df

,other_created_address,date_creation_diff,verified,tx_hash,initiator,gas,gas_price
0,0xfa7d2a996ac6350f4b56c043112da0366a59b74c,438,1,0x1c6c4cd001e65bb249a4aacf1a8551b39cba271fb91f...,0x78655080b65f42e2cee5fa5673689cc44d4e1cfc,401230,50000000000
1,0x575ccd8e2d300e2377b43478339e364000318e2c,0,1,0x1fd143ee75c6be745c51866e79f83c0311ae02180830...,0xc4ad0ef33a0a4dda3461c479ccb6c36d1e4b7be4,975747,220000000000
2,0x705350c4bcd35c9441419ddd5d2f097d7a55410f,-1,1,0x6d11ec6a195a4f9d968f1f8c0deaebb8ecbdae2a46e0...,0xc4ad0ef33a0a4dda3461c479ccb6c36d1e4b7be4,1921440,280000000000
3,0xa90996896660decc6e997655e065b23788857849,-1,1,0x8545a0f97e2f228b875e2d67402c44cc6e4ca0a2042a...,0xc4ad0ef33a0a4dda3461c479ccb6c36d1e4b7be4,1921440,280000000000
4,0xb1f2cdec61db658f091671f5f199635aef202cac,-1,1,0x96a266744df0e60ed734dad2e8a8f4eeaef8454ce64b...,0xc4ad0ef33a0a4dda3461c479ccb6c36d1e4b7be4,1418783,280000000000
...,...,...,...,...,...,...,...
731,0xeee72c5da95a1e4bbcf0bf327518045ff5a2916f,149,0,0x1c62141c949df01ee37d9145f7cea87904b3db84c020...,0x55a0e991015d9d00067f6dfca8c0e1bebef68cd5,2562844,136000000000
732,0xca658217ce94dfb2156a49a8fad0ff752cac39c2,59,0,0x4d18e286dffa967b7937499513d7781668af474de366...,0x55a0e991015d9d00067f6dfca8c0e1bebef68cd5,2794383,81000000000
733,0x06a00715e6f92210af9d7680b584931faf71a833,0,1,0x83273787890870588c4689d8c70d8eca46bc560cd23f...,0x85c03354e4c708e0d93d442eef9e4b3da1d713dd,2013399,71953123116
734,0xcafe001067cdef266afb7eb5a286dcfd277f3de5,0,1,0xad25be756c3dc12db179548dee5f7dbb07d4c763aa5d...,0x490ce4616672e93b1c8f5e43aa80312fd73dee8c,2859074,112131911642


In [16]:
ml_data_all = pd.merge(ml_data[['other_created_address','last_tx_in_days']], gas_feature_df, on='other_created_address', how='left')
# Move column 'B' to the last position
col = ml_data_all.pop('verified')
ml_data_all.insert(len(ml_data_all.columns), 'verified', col)

ml_data_all.to_csv('./raw_data/ml_data.csv')

                          other_created_address  last_tx_in_days  \
0    0xfa7d2a996ac6350f4b56c043112da0366a59b74c               65   
1    0x575ccd8e2d300e2377b43478339e364000318e2c                2   
2    0x705350c4bcd35c9441419ddd5d2f097d7a55410f               11   
3    0xa90996896660decc6e997655e065b23788857849                2   
4    0xb1f2cdec61db658f091671f5f199635aef202cac               -1   
..                                          ...              ...   
731  0xeee72c5da95a1e4bbcf0bf327518045ff5a2916f              720   
732  0xca658217ce94dfb2156a49a8fad0ff752cac39c2                3   
733  0x06a00715e6f92210af9d7680b584931faf71a833                0   
734  0xcafe001067cdef266afb7eb5a286dcfd277f3de5               -1   
735  0xf8e9f10c22840b613cda05a0c5fdb59a4d6cd7ef               -1   

     date_creation_diff                                            tx_hash  \
0                   438  0x1c6c4cd001e65bb249a4aacf1a8551b39cba271fb91f...   
1                     0  0x